### Preparing Working Env

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from importlib.util import find_spec
if find_spec("core") is None:
    import sys
    sys.path.append('..')

import tensorflow as tf
import tensorflow_datasets as tfds
from core.datasets import RetinaDataset

In [2]:
#This code snippet helps if your computer has RTX 2070 GPU. If not then comment this cell.
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

### Preparing the Data

In [3]:
ds_train, ds_train_info = tfds.load('RetinaDataset', split='train[:98%]', shuffle_files=True, as_supervised=True,with_info=True)
ds_val, ds_val_info     = tfds.load('RetinaDataset', split='train[-2%:]', shuffle_files=True, as_supervised=True,with_info=True)
ds_test, ds_test_info   = tfds.load('RetinaDataset', split='test', shuffle_files=True, as_supervised=True,with_info=True)

### SimCLR Data Utils

In [5]:
#@title SimCLR DataUtils

# coding=utf-8
# Copyright 2020 The SimCLR Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific simclr governing permissions and
# limitations under the License.
# ==============================================================================
"""Data preprocessing and augmentation."""

import functools
from absl import flags

import tensorflow.compat.v2 as tf

FLAGS = flags.FLAGS

CROP_PROPORTION = 0.875  # Standard for ImageNet.


def random_apply(func, p, x):
    """Randomly apply function func to x with probability p."""
    return tf.cond(
      tf.less(
          tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
          tf.cast(p, tf.float32)), lambda: func(x), lambda: x)


def random_brightness(image, max_delta, impl='simclrv2'):
    """A multiplicative vs additive change of brightness."""
    if impl == 'simclrv2':
        factor = tf.random.uniform([], tf.maximum(1.0 - max_delta, 0),
                                   1.0 + max_delta)
        image = image * factor
    elif impl == 'simclrv1':
        image = tf.image.random_brightness(image, max_delta=max_delta)
    else:
        raise ValueError('Unknown impl {} for random brightness.'.format(impl))
    return image


def to_grayscale(image, keep_channels=True):
    image = tf.image.rgb_to_grayscale(image)
    if keep_channels:
        image = tf.tile(image, [1, 1, 3])
    return image


def color_jitter(image, strength, random_order=True, impl='simclrv2'):
    """Distorts the color of the image.
    Args:
    image: The input image tensor.
    strength: the floating number for the strength of the color augmentation.
    random_order: A bool, specifying whether to randomize the jittering order.
    impl: 'simclrv1' or 'simclrv2'.  Whether to use simclrv1 or simclrv2's
        version of random brightness.
    Returns:
    The distorted image tensor.
    """
    brightness = 0.8 * strength
    contrast = 0.8 * strength
    saturation = 0.8 * strength
    hue = 0.2 * strength
    if random_order:
        return color_jitter_rand(
            image, brightness, contrast, saturation, hue, impl=impl)
    else:
        return color_jitter_nonrand(
            image, brightness, contrast, saturation, hue, impl=impl)


def color_jitter_nonrand(image,
                         brightness=0,
                         contrast=0,
                         saturation=0,
                         hue=0,
                         impl='simclrv2'):
    """Distorts the color of the image (jittering order is fixed).
    Args:
    image: The input image tensor.
    brightness: A float, specifying the brightness for color jitter.
    contrast: A float, specifying the contrast for color jitter.
    saturation: A float, specifying the saturation for color jitter.
    hue: A float, specifying the hue for color jitter.
    impl: 'simclrv1' or 'simclrv2'.  Whether to use simclrv1 or simclrv2's
        version of random brightness.
    Returns:
    The distorted image tensor.
    """
    with tf.name_scope('distort_color'):
        def apply_transform(i, x, brightness, contrast, saturation, hue):
            """Apply the i-th transformation."""
            if brightness != 0 and i == 0:
                x = random_brightness(x, max_delta=brightness, impl=impl)
            elif contrast != 0 and i == 1:
                x = tf.image.random_contrast(
                    x, lower=1-contrast, upper=1+contrast)
            elif saturation != 0 and i == 2:
                x = tf.image.random_saturation(
                    x, lower=1-saturation, upper=1+saturation)
            elif hue != 0:
                x = tf.image.random_hue(x, max_delta=hue)
            return x

        for i in range(4):
            image = apply_transform(i, image, brightness, contrast, saturation, hue)
        return image


def color_jitter_rand(image,
                      brightness=0,
                      contrast=0,
                      saturation=0,
                      hue=0,
                      impl='simclrv2'):
    """Distorts the color of the image (jittering order is random).
    Args:
    image: The input image tensor.
    brightness: A float, specifying the brightness for color jitter.
    contrast: A float, specifying the contrast for color jitter.
    saturation: A float, specifying the saturation for color jitter.
    hue: A float, specifying the hue for color jitter.
    impl: 'simclrv1' or 'simclrv2'.  Whether to use simclrv1 or simclrv2's
        version of random brightness.
    Returns:
    The distorted image tensor.
    """
    with tf.name_scope('distort_color'):
        def apply_transform(i, x):
            """Apply the i-th transformation."""
            def brightness_foo():
                if brightness == 0:
                    return x
                else:
                    return random_brightness(x, max_delta=brightness, impl=impl)

            def contrast_foo():
                if contrast == 0:
                    return x
                else:
                    return tf.image.random_contrast(x, lower=1-contrast, upper=1+contrast)
            def saturation_foo():
                if saturation == 0:
                    return x
                else:
                    return tf.image.random_saturation(
                      x, lower=1-saturation, upper=1+saturation)
            def hue_foo():
                if hue == 0:
                    return x
                else:
                    return tf.image.random_hue(x, max_delta=hue)
            x = tf.cond(tf.less(i, 2),
                  lambda: tf.cond(tf.less(i, 1), brightness_foo, contrast_foo),
                  lambda: tf.cond(tf.less(i, 3), saturation_foo, hue_foo))
            return x

        perm = tf.random.shuffle(tf.range(4))
        for i in range(4):
            image = apply_transform(perm[i], image)
        return image


def _compute_crop_shape(
    image_height, image_width, aspect_ratio, crop_proportion):
    """Compute aspect ratio-preserving shape for central crop.
    The resulting shape retains `crop_proportion` along one side and a proportion
    less than or equal to `crop_proportion` along the other side.
    Args:
    image_height: Height of image to be cropped.
    image_width: Width of image to be cropped.
    aspect_ratio: Desired aspect ratio (width / height) of output.
    crop_proportion: Proportion of image to retain along the less-cropped side.
    Returns:
    crop_height: Height of image after cropping.
    crop_width: Width of image after cropping.
    """
    image_width_float = tf.cast(image_width, tf.float32)
    image_height_float = tf.cast(image_height, tf.float32)

    def _requested_aspect_ratio_wider_than_image():
        crop_height = tf.cast(
            tf.math.rint(crop_proportion / aspect_ratio * image_width_float),
            tf.int32)
        crop_width = tf.cast(
            tf.math.rint(crop_proportion * image_width_float), tf.int32)
        return crop_height, crop_width

    def _image_wider_than_requested_aspect_ratio():
        crop_height = tf.cast(
            tf.math.rint(crop_proportion * image_height_float), tf.int32)
        crop_width = tf.cast(
            tf.math.rint(crop_proportion * aspect_ratio * image_height_float),
            tf.int32)
        return crop_height, crop_width

    return tf.cond(
      aspect_ratio > image_width_float / image_height_float,
      _requested_aspect_ratio_wider_than_image,
      _image_wider_than_requested_aspect_ratio)


def center_crop(image, height, width, crop_proportion):
    """Crops to center of image and rescales to desired size.
    Args:
    image: Image Tensor to crop.
    height: Height of image to be cropped.
    width: Width of image to be cropped.
    crop_proportion: Proportion of image to retain along the less-cropped side.
    Returns:
    A `height` x `width` x channels Tensor holding a central crop of `image`.
    """
    shape = tf.shape(image)
    image_height = shape[0]
    image_width = shape[1]
    crop_height, crop_width = _compute_crop_shape(
      image_height, image_width, height / width, crop_proportion)
    offset_height = ((image_height - crop_height) + 1) // 2
    offset_width = ((image_width - crop_width) + 1) // 2
    image = tf.image.crop_to_bounding_box(
      image, offset_height, offset_width, crop_height, crop_width)

    image = tf.image.resize([image], [height, width],
                          method=tf.image.ResizeMethod.BICUBIC)[0]

    return image


def distorted_bounding_box_crop(image,
                                bbox,
                                min_object_covered=0.1,
                                aspect_ratio_range=(0.75, 1.33),
                                area_range=(0.05, 1.0),
                                max_attempts=100,
                                scope=None):
    """Generates cropped_image using one of the bboxes randomly distorted.
    See `tf.image.sample_distorted_bounding_box` for more documentation.
    Args:
    image: `Tensor` of image data.
    bbox: `Tensor` of bounding boxes arranged `[1, num_boxes, coords]`
        where each coordinate is [0, 1) and the coordinates are arranged
        as `[ymin, xmin, ymax, xmax]`. If num_boxes is 0 then use the whole
        image.
    min_object_covered: An optional `float`. Defaults to `0.1`. The cropped
        area of the image must contain at least this fraction of any bounding
        box supplied.
    aspect_ratio_range: An optional list of `float`s. The cropped area of the
        image must have an aspect ratio = width / height within this range.
    area_range: An optional list of `float`s. The cropped area of the image
        must contain a fraction of the supplied image within in this range.
    max_attempts: An optional `int`. Number of attempts at generating a cropped
        region of the image of the specified constraints. After `max_attempts`
        failures, return the entire image.
    scope: Optional `str` for name scope.
    Returns:
    (cropped image `Tensor`, distorted bbox `Tensor`).
    """
    with tf.name_scope(scope or 'distorted_bounding_box_crop'):
        shape = tf.shape(image)
        sample_distorted_bounding_box = tf.image.sample_distorted_bounding_box(
            shape,
            bounding_boxes=bbox,
            min_object_covered=min_object_covered,
            aspect_ratio_range=aspect_ratio_range,
            area_range=area_range,
            max_attempts=max_attempts,
            use_image_if_no_bounding_boxes=True)
        bbox_begin, bbox_size, _ = sample_distorted_bounding_box

        # Crop the image to the specified bounding box.
        offset_y, offset_x, _ = tf.unstack(bbox_begin)
        target_height, target_width, _ = tf.unstack(bbox_size)
        image = tf.image.crop_to_bounding_box(
            image, offset_y, offset_x, target_height, target_width)

        return image


def crop_and_resize(image, height, width):
    """Make a random crop and resize it to height `height` and width `width`.
    Args:
    image: Tensor representing the image.
    height: Desired image height.
    width: Desired image width.
    Returns:
    A `height` x `width` x channels Tensor holding a random crop of `image`.
    """
    bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])
    aspect_ratio = width / height
    image = distorted_bounding_box_crop(
      image,
      bbox,
      min_object_covered=0.1,
      aspect_ratio_range=(3. / 4 * aspect_ratio, 4. / 3. * aspect_ratio),
      area_range=(0.08, 1.0),
      max_attempts=100,
      scope=None)
    return tf.image.resize([image], [height, width],
                         method=tf.image.ResizeMethod.BICUBIC)[0]


def gaussian_blur(image, kernel_size, sigma, padding='SAME'):
    """Blurs the given image with separable convolution.
    Args:
    image: Tensor of shape [height, width, channels] and dtype float to blur.
    kernel_size: Integer Tensor for the size of the blur kernel. This is should
      be an odd number. If it is an even number, the actual kernel size will be
      size + 1.
    sigma: Sigma value for gaussian operator.
    padding: Padding to use for the convolution. Typically 'SAME' or 'VALID'.
    Returns:
    A Tensor representing the blurred image.
    """
    radius = tf.cast(kernel_size / 2, dtype=tf.int32)
    kernel_size = radius * 2 + 1
    x = tf.cast(tf.range(-radius, radius + 1), dtype=tf.float32)
    blur_filter = tf.exp(-tf.pow(x, 2.0) /
                       (2.0 * tf.pow(tf.cast(sigma, dtype=tf.float32), 2.0)))
    blur_filter /= tf.reduce_sum(blur_filter)
    # One vertical and one horizontal filter.
    blur_v = tf.reshape(blur_filter, [kernel_size, 1, 1, 1])
    blur_h = tf.reshape(blur_filter, [1, kernel_size, 1, 1])
    num_channels = tf.shape(image)[-1]
    blur_h = tf.tile(blur_h, [1, 1, num_channels, 1])
    blur_v = tf.tile(blur_v, [1, 1, num_channels, 1])
    expand_batch_dim = image.shape.ndims == 3
    if expand_batch_dim:
        # Tensorflow requires batched input to convolutions, which we can fake with
        # an extra dimension.
        image = tf.expand_dims(image, axis=0)
    blurred = tf.nn.depthwise_conv2d(
      image, blur_h, strides=[1, 1, 1, 1], padding=padding)
    blurred = tf.nn.depthwise_conv2d(
      blurred, blur_v, strides=[1, 1, 1, 1], padding=padding)
    if expand_batch_dim:
        blurred = tf.squeeze(blurred, axis=0)
    return blurred


def random_crop_with_resize(image, height, width, p=1.0):
    """Randomly crop and resize an image.
    Args:
    image: `Tensor` representing an image of arbitrary size.
    height: Height of output image.
    width: Width of output image.
    p: Probability of applying this transformation.
    Returns:
    A preprocessed image `Tensor`.
    """
    def _transform(image):  # pylint: disable=missing-docstring
        image = crop_and_resize(image, height, width)
        return image
    return random_apply(_transform, p=p, x=image)


def random_color_jitter(image, p=1.0, impl='simclrv2', color_jitter_strength=0.8):

    def _transform(image):
        color_jitter_t = functools.partial(
            color_jitter, strength=color_jitter_strength, impl=impl)
        image = random_apply(color_jitter_t, p=0.8, x=image)
        return random_apply(to_grayscale, p=0.2, x=image)
    return random_apply(_transform, p=p, x=image)


def random_blur(image, height, width, p=1.0):
    """Randomly blur an image.
    Args:
    image: `Tensor` representing an image of arbitrary size.
    height: Height of output image.
    width: Width of output image.
    p: probability of applying this transformation.
    Returns:
    A preprocessed image `Tensor`.
    """
    del width
    def _transform(image):
        sigma = tf.random.uniform([], 0.1, 2.0, dtype=tf.float32)
        return gaussian_blur(image, kernel_size=height//10, sigma=sigma, padding='SAME')
    return random_apply(_transform, p=p, x=image)


def batch_random_blur(images_list, height, width, blur_probability=0.5):
    """Apply efficient batch data transformations.
    Args:
    images_list: a list of image tensors.
    height: the height of image.
    width: the width of image.
    blur_probability: the probaility to apply the blur operator.
    Returns:
    Preprocessed feature list.
    """
    def generate_selector(p, bsz):
        shape = [bsz, 1, 1, 1]
        selector = tf.cast(
            tf.less(tf.random.uniform(shape, 0, 1, dtype=tf.float32), p),
            tf.float32)
        return selector

    new_images_list = []
    for images in images_list:
        images_new = random_blur(images, height, width, p=1.)
        selector = generate_selector(blur_probability, tf.shape(images)[0])
        images = images_new * selector + images * (1 - selector)
        images = tf.clip_by_value(images, 0., 1.)
        new_images_list.append(images)

    return new_images_list


def preprocess_for_train(image,
                         height,
                         width,
                         color_distort=True,
                         crop=True,
                         flip=True,
                         color_jitter_strength=0.9,
                         impl='simclrv2'):
    """Preprocesses the given image for training.
    Args:
    image: `Tensor` representing an image of arbitrary size.
    height: Height of output image.
    width: Width of output image.
    color_distort: Whether to apply the color distortion.
    crop: Whether to crop the image.
    flip: Whether or not to flip left and right of an image.
    impl: 'simclrv1' or 'simclrv2'.  Whether to use simclrv1 or simclrv2's
        version of random brightness.
    Returns:
    A preprocessed image `Tensor`.
    """
    if crop:
        image = random_crop_with_resize(image, height, width)
    if flip:
        image = tf.image.random_flip_left_right(image)
    if color_distort:
        image = random_color_jitter(image, impl=impl, 
                                    color_jitter_strength=color_jitter_strength)
    image = tf.reshape(image, [height, width, 3])
    return image


def preprocess_for_eval(image, height, width, crop=True):
    """Preprocesses the given image for evaluation.
    Args:
    image: `Tensor` representing an image of arbitrary size.
    height: Height of output image.
    width: Width of output image.
    crop: Whether or not to (center) crop the test images.
    Returns:
    A preprocessed image `Tensor`.
    """
    if crop:
        image = center_crop(image, height, width, crop_proportion=CROP_PROPORTION)
    image = tf.reshape(image, [height, width, 3])
    # image = tf.clip_by_value(image, 0., 1.)
    return image


def preprocess_image(image, height, width, is_training=False,
                     color_distort=True, test_crop=True):
    """Preprocesses the given image.
    Args:
    image: `Tensor` representing an image of arbitrary size.
    height: Height of output image.
    width: Width of output image.
    is_training: `bool` for whether the preprocessing is for training.
    color_distort: whether to apply the color distortion.
    test_crop: whether or not to extract a central crop of the images
        (as for standard ImageNet evaluation) during the evaluation.
    Returns:
    A preprocessed image `Tensor` of range [0, 1].
    """
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    if is_training:
        return preprocess_for_train(image, height, width, color_distort)
    else:
        return preprocess_for_eval(image, height, width, test_crop)

In [6]:
from functools import partial

NCLASS   = 4
IMG_SIZE = 224

def resize_image(img, lb):
  return tf.image.resize(img, (224,224)), tf.one_hot(lb, NCLASS)

def augment_image(img, lb):
  img, lb = resize_image(img, lb)
  return preprocess_for_train(img, height=IMG_SIZE, width=IMG_SIZE), lb

ds_train_augment = ds_train.map(augment_image)
ds_val = ds_val.map(resize_image)

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.


Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.


### Computing Weight Classes

In [7]:
y_labels = []
labels = ds_train_augment.map(lambda x, y: y)
for l in labels.batch(64).as_numpy_iterator():
  y_labels.append(l)
y_labels = np.vstack(y_labels)
y_labels.sum(axis=0)

array([25775.,  8434., 11112., 36498.], dtype=float32)

In [8]:
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced', [0, 1, 2, 3], y_labels.argmax(axis=1))
class_weights = {i: w for i, w in enumerate(class_weights)}
print(class_weights)

{0: 0.7935887487875849, 1: 2.4252727057149635, 2: 1.8407802375809936, 3: 0.5604348183462108}


C:\Users\mohit\Anaconda3\envs\retina_env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0, 1, 2, 3], y=[3 3 3 ... 3 0 3] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


### Creating Classifier with ResNet Base

In [9]:
def create_basemodel(base_model='resnet', 
                     input_shape=(224,224,3),
                     output_units=4):
  
    if base_model == 'resnet':
        preprocess = tf.keras.applications.resnet_v2.preprocess_input
        base_model = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet')
    elif base_model == 'xception':
        preprocess = tf.keras.applications.xception.preprocess_input
        base_model = tf.keras.applications.Xception(include_top=False, weights='imagenet')
    elif base_model == 'inception':
        preprocess = tf.keras.layers.Lambda(lambda x: x)
        base_model = tf.kaers.applications.inception_v3.InceptionV3(include_top=False, weights='imagenet')
    else:
        raise f"{base_model} not supported, choose from ['resnet', 'xception', 'inception']"

    base_model.trainable = False

    inputs = tf.keras.layers.Input(input_shape)
    pool    = tf.keras.layers.GlobalAveragePooling2D()
    flatten = tf.keras.layers.Flatten()

    softmax   = tf.keras.layers.Dense(output_units, activation='softmax')

    x = inputs
    x = preprocess(x)
    x = base_model(x)
    x = tf.keras.layers.Conv2D(64, 3, activation="relu")(x)
    x = pool(x)
    x = flatten(x)
    out = softmax(x)

    return tf.keras.Model(inputs=inputs, outputs=out)

In [10]:
resnet_base = create_basemodel('resnet')

metrics = ['accuracy']
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', ),
             tf.keras.callbacks.ModelCheckpoint(filepath='resnet_model.{epoch:02d}-{val_loss:.2f}.h5'),]
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
resnet_base.compile(optimizer=optimizer, 
                    loss='categorical_crossentropy', 
                    metrics=metrics)

In [11]:
resnet_base.fit(ds_train_augment.batch(32),
                validation_data = ds_val.batch(32),
                callbacks = callbacks,
                class_weight = class_weights, 
                epochs=20)

Epoch 1/20
2557/2557 [==============================] - 162s 63ms/step - loss: 0.7330 - accuracy: 0.7402 - val_loss: 0.4418 - val_accuracy: 0.8431
Epoch 2/20
2557/2557 [==============================] - 162s 63ms/step - loss: 0.6188 - accuracy: 0.7797 - val_loss: 0.5046 - val_accuracy: 0.8162
Epoch 3/20
2557/2557 [==============================] - 163s 64ms/step - loss: 0.5961 - accuracy: 0.7870 - val_loss: 0.4796 - val_accuracy: 0.8275
Epoch 4/20
2557/2557 [==============================] - 163s 64ms/step - loss: 0.5745 - accuracy: 0.7928 - val_loss: 0.4796 - val_accuracy: 0.8275


In [23]:
# %ls

In [13]:
refine_resnet_base = create_basemodel('resnet')
refine_resnet_base.load_weights('resnet_model.01-0.44.h5')

In [14]:
metrics = ['accuracy']
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
refine_resnet_base.compile(optimizer=optimizer, 
                           loss='categorical_crossentropy', 
                           metrics=metrics)

refine_resnet_base.evaluate(ds_val.batch(32))

53/53 [==============================] - 3s 57ms/step - loss: 0.4418 - accuracy: 0.8431


[0.4418461322784424, 0.8431137800216675]

### Fine-tuning the Classifier

In [15]:
refine_resnet_base.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv_1 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_Sub_1 (TensorFlo [(None, 224, 224, 3)]     0         
_________________________________________________________________
resnet50v2 (Functional)      (None, None, None, 2048)  23564800  
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 5, 5, 64)          1179712   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 64)                0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)               

In [16]:
# Extract the base model
base_model = refine_resnet_base.layers[3]

# unfreeze it
base_model.trainable = True

# select only the last resnet block for retraining
# keeping the batchnorm layer unchanged
for l in base_model.layers:
  name = l.name
  if name.startswith('conv5_block3') and not isinstance(l, tf.keras.layers.BatchNormalization):
    l.trainable = True
  else:
    l.trainable = False

optimizer = tf.keras.optimizers.Adam(lr=1e-4)
refine_resnet_base.compile(optimizer=optimizer, 
                           loss='categorical_crossentropy', 
                           metrics=metrics)

In [17]:
callbacks = [tf.keras.callbacks.EarlyStopping(patience=3, monitor='val_loss', ),
             tf.keras.callbacks.ModelCheckpoint(filepath='finetune_resnet_model.{epoch:02d}-{val_loss:.2f}.h5'),]

refine_resnet_base.fit(ds_train_augment.batch(32), 
                       validation_data = ds_val.batch(32),
                       callbacks = callbacks,
                       class_weight = class_weights, 
                       epochs=20)

Epoch 1/20
2557/2557 [==============================] - 182s 71ms/step - loss: 0.5576 - accuracy: 0.8046 - val_loss: 0.3581 - val_accuracy: 0.8772
Epoch 2/20
2557/2557 [==============================] - 182s 71ms/step - loss: 0.4936 - accuracy: 0.8287 - val_loss: 0.3592 - val_accuracy: 0.8713
Epoch 3/20
2557/2557 [==============================] - 183s 72ms/step - loss: 0.4650 - accuracy: 0.8368 - val_loss: 0.3329 - val_accuracy: 0.8868
Epoch 4/20
2557/2557 [==============================] - 183s 72ms/step - loss: 0.4449 - accuracy: 0.8447 - val_loss: 0.2759 - val_accuracy: 0.9036
Epoch 5/20
2557/2557 [==============================] - 183s 72ms/step - loss: 0.4362 - accuracy: 0.8462 - val_loss: 0.3024 - val_accuracy: 0.8880
Epoch 6/20
2557/2557 [==============================] - 183s 71ms/step - loss: 0.4257 - accuracy: 0.8506 - val_loss: 0.3168 - val_accuracy: 0.8880
Epoch 7/20
2557/2557 [==============================] - 183s 71ms/step - loss: 0.4151 - accuracy: 0.8549 - val_loss: 0

In [22]:
# %ls

### Selecting the Best Model

In [19]:
refine_resnet_base.load_weights('finetune_resnet_model.08-0.25.h5')

In [20]:
refine_resnet_base.evaluate(ds_test.map(resize_image).batch(32))

31/31 [==============================] - 3s 91ms/step - loss: 0.0358 - accuracy: 0.9928


[0.03580612689256668, 0.9927685856819153]

In [21]:
refine_resnet_base.evaluate(ds_val.batch(32))

53/53 [==============================] - 4s 80ms/step - loss: 0.2524 - accuracy: 0.9072


[0.2524038255214691, 0.9071856141090393]

In [24]:
from core.models.base import WEIGHTS_DIRNAME

In [26]:
wt_path = str(WEIGHTS_DIRNAME)+"/resnet_with_conv.h5"
wt_path = wt_path.replace('\\','/')

In [27]:
refine_resnet_base.save_weights(wt_path)

In [32]:
from core.networks.resnet_with_conv_finetune import resnetconvfinetune

In [33]:
test_model = resnetconvfinetune(input_shape = (224, 224, 3), output_shape = (4,))
test_model.load_weights(wt_path)

In [35]:
optimizer = tf.keras.optimizers.Adam(lr=1e-4)
test_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=metrics)
test_model.evaluate(ds_test.map(resize_image).batch(32))

31/31 [==============================] - 3s 82ms/step - loss: 0.0358 - accuracy: 0.9928 0s - loss: 0.0382 - ac


[0.03580612689256668, 0.9927685856819153]

#### END